# Layerwise learning for Quantum Neural Nets

In [1]:
import random
import collections
import matplotlib.pyplot as plt

# Pennylane
import pennylane as qml
from pennylane import numpy as np

# Pytorch
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

In [2]:
torch.cuda.is_available()

False

### Parameters

In [3]:
n_qubits = 9
n_quantum_layers = 2
batch_size = 4
epochs = 1

### Data pre-processing

In [4]:
data_transforms = transforms.Compose([transforms.Resize(28), #resize to a 28x28 image
                                      transforms.ToTensor(), #convert to tensor
                                      transforms.Lambda(lambda x: torch.flatten(x)) #obtain a one dimensional vector
                                     ])

In [5]:
train_set = datasets.MNIST(root='./data', train=True, download=True, transform=data_transforms)
test_set = datasets.MNIST(root='./data', train=False, download=True, transform=data_transforms)

train_set.targets[train_set.targets == 1] = 10
train_set.targets[train_set.targets == 0] = 10
train_set.targets[train_set.targets == 3] = 0
train_set.targets[train_set.targets == 6] = 1

print(train_set.targets.unique())

# Filter to just images of '3's and '6's
subset_indices = ((train_set.targets == -1) + (train_set.targets == 1)).nonzero().view(-1)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=False,
                                          sampler=SubsetRandomSampler(subset_indices))
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False,
                                         sampler=SubsetRandomSampler(subset_indices))

tensor([ 0,  1,  2,  4,  5,  7,  8,  9, 10])


In [6]:
dataiter = iter(train_loader)
images, labels = dataiter.next()

print(images.shape)
print(labels[3])

torch.Size([4, 784])
tensor(1)


In [7]:
dev = qml.device("default.qubit", wires=n_qubits)

In [8]:
def set_random_gates(n_qubits: int):
    
    gate_set = [qml.RX, qml.RY, qml.RZ]
    chosen_gates = []
    for i in range(n_qubits):
        chosen_gate = random.choice(gate_set)
        chosen_gates.append(chosen_gate)
    return chosen_gates

### Variational circuit

We first define some quantum layers that will compose the quantum circuit.

In [9]:
def apply_layer(gates, weights):
    """Docstrings"""
    
    for i in range(n_qubits): 
        gates[i](weights[i], wires = i)
    
    tuples = [(i,i+1) for i in range(n_qubits-1)]

    for tup in tuples:
        qml.CZ(wires=[tup[0], tup[1]])

In [10]:
@qml.qnode(dev, interface="torch")
def quantum_net(inputs, layer_weights):
    """Docstrings"""
        
    wirelist = [i for i in range(n_qubits)]
    
    #Encode the data with Angle Embedding
    qml.templates.AngleEmbedding(inputs, wires=wirelist, rotation='X')
    
    # Sequence of trainable layers
    for i in range(len(layer_gates)):
        apply_layer(layer_gates[i], layer_weights[i]) #maybe "weights" needs a reformat
        
    # Expectation value of the last qubit
    return qml.expval(qml.PauliZ(n_qubits-2)), qml.expval(qml.PauliZ(n_qubits-1))

In [11]:
# integrate it as a TorchLayer
# https://pennylane.readthedocs.io/en/stable/code/api/pennylane.qnn.TorchLayer.html

In [12]:
# Obtain random gates for each layer
layer_gates = [set_random_gates(n_qubits) for i in range(n_quantum_layers)]

# Define shape of the weights
weight_shapes = {"layer_weights": (n_quantum_layers, n_qubits)}

In [13]:
qlayer = qml.qnn.TorchLayer(quantum_net, weight_shapes, init_method = nn.init.zeros_)
layer_prenet = nn.Linear(784, n_qubits)
softmax = torch.nn.Softmax(dim=1)
layer_postnet = nn.Linear(2, 2)

model = torch.nn.Sequential(layer_prenet, qlayer, layer_postnet, softmax)

In [14]:
opt = optim.Adam(model.parameters(), lr=0.01)

loss = nn.CrossEntropyLoss()

In [ ]:
for epoch in range(epochs):
    print("hola")
    for x, y in train_loader:
        opt.zero_grad()
        loss_evaluated = loss(model(x), y)
        print("hola")
        loss_evaluated.backward()

        opt.step()
    print("Average loss over epoch {}: {:.4f}".format(epoch + 1, loss_evaluated))


hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola
hola


In [ ]:
import sklearn.datasets

In [ ]:
n_qubits = 2
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def qnode(inputs, weights):
    qml.templates.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.templates.StronglyEntanglingLayers(weights, wires=range(n_qubits))
    return qml.expval(qml.PauliZ(0)), qml.expval(qml.PauliZ(1))

weight_shapes = {"weights": (3, n_qubits, 3)}

qlayer = qml.qnn.TorchLayer(qnode, weight_shapes)
clayer1 = torch.nn.Linear(2, 2)
clayer2 = torch.nn.Linear(2, 2)
softmax = torch.nn.Softmax(dim=1)
model = torch.nn.Sequential(clayer1, qlayer, clayer2, softmax)

samples = 100
x, y = sklearn.datasets.make_moons(samples)
y_hot = np.zeros((samples, 2))
y_hot[np.arange(samples), y] = 1

X = torch.tensor(x).float()

print(X.shape)
print(Y.shape)

opt = optim.Adam(model.parameters(), lr=0.01)
loss = torch.nn.CrossEntropyLoss()

In [ ]:
epochs = 8
batch_size = 5
batches = samples // batch_size

data_loader = torch.utils.data.DataLoader(list(zip(X, y)), batch_size=batch_size,
                                          shuffle=True, drop_last=True)

for epoch in range(epochs):

    running_loss = 0

    for x, y in data_loader:
        opt.zero_grad()

        loss_evaluated = loss(model(x), y)
        loss_evaluated.backward()

        opt.step()

        running_loss += loss_evaluated

    avg_loss = running_loss / batches
    print("Average loss over epoch {}: {:.4f}".format(epoch + 1, avg_loss))

In [ ]:
n_qubits = 2
n_quantum_layers = 4

In [ ]:
@qml.qnode(dev, interface="torch")
def quantum_net(inputs, layer_weights):
    """Docstrings"""
        
    wirelist = [i for i in range(n_qubits)]
    
    #Encode the data with Angle Embedding
    qml.templates.AngleEmbedding(inputs, wires=wirelist, rotation='X')
    
    # Sequence of trainable layers
    for i in range(len(layer_gates)):
        apply_layer(layer_gates[i], layer_weights[i]) #maybe "weights" needs a reformat
        
    # Expectation value of the last qubit
    return qml.expval(qml.PauliZ(n_qubits-2)), qml.expval(qml.PauliZ(n_qubits-1))

In [ ]:
# Obtain random gates for each layer
layer_gates = [set_random_gates(n_qubits) for i in range(n_quantum_layers)]

# Define shape of the weights
weight_shapes = {"layer_weights": (n_quantum_layers, n_qubits)}

In [ ]:
qlayer = qml.qnn.TorchLayer(quantum_net, weight_shapes, init_method = nn.init.zeros_)
clayer1 = torch.nn.Linear(2, 2)
clayer2 = torch.nn.Linear(2, 2)
softmax = torch.nn.Softmax(dim=1)
model = torch.nn.Sequential(clayer1, qlayer, clayer2, softmax)

In [ ]:
opt = optim.Adam(model.parameters(), lr=0.01)
loss = torch.nn.L1Loss()

In [ ]:
epochs = 8
batch_size = 5
batches = samples // batch_size

data_loader = torch.utils.data.DataLoader(list(zip(X, Y)), batch_size=batch_size,
                                          shuffle=True, drop_last=True)

for epoch in range(epochs):

    running_loss = 0

    for x, y in data_loader:
        opt.zero_grad()

        loss_evaluated = loss(model(x), y)
        loss_evaluated.backward()

        opt.step()

        running_loss += loss_evaluated

    avg_loss = running_loss / batches
    print("Average loss over epoch {}: {:.4f}".format(epoch + 1, avg_loss))